# Projeto beAnalytic

Nesse projeto tenho a missão de realizar a extração das informações da base de dados listada no website:

https://steamdb.info/sales/

Além disso um tratamento de dados será efetuado.

In [1]:
import requests

# URL da API
url = 'https://steamdb.info/sales/'

res = requests.get(url)

print(res)

<Response [403]>


O primeiro passo para a obtenção dos dados não teve êxito. A página alvo tem uma proteção contra web scraping, o que impossibilita a solicitação de dados. De acordo com o FAQ da página, essa prática pode levar a um banimento e, além disso, eles não disponibilizam uma API.

Diante dessa situação, adotei uma segunda estratégia para cumprir o exercício proposto. Mediante a inspeção da página, foi possível copiar a seção referente à tabela como um arquivo HTML e salvá-lo localmente. Na página, aplicamos um filtro para obter dados de 5 mil jogos. Assim, conseguimos acessar as informações e extrair os dados desse arquivo.

Dessa forma, foi possível prosseguir com a análise normalmente, pois o algoritmo que implementado poderia ser aplicado da mesma forma se a solicitação de dados tivesse funcionado.

In [2]:
# Bibliotecas
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

html_data = open('steamDB_5000.html',encoding="utf8")

soup = BeautifulSoup(html_data, 'html.parser')

In [3]:
# Averiguando quantos jogos temos no total, para isso vamos usar o método .find() para localizar a tag <a> com a classe "b",
# que se refere ao nome do jogo.
qtd = soup.findAll('a', class_='b')
print('total :',len(qtd))

total : 4707


In [4]:
# Criando uma lista com as informações de cada jogo

# Encontre todas as tags 'a' com a classe 'b'
names = soup.find_all('a', class_='b')

# Criando uma lista para armazenar todos os dicionários
data = []

# Para cada tag 'a' extraio as informações desejadas
for name in names:
    # Criando um dicionário para armazenar as informações
    dic = {}

    # Buscando a tag div com classe 'subinfo' (que pode não existir)
    div = name.find_next('div', class_='subinfo')

    # Buscando as tags <span> dentro da div (que podem não existir)
    if div:
        free_tag = div.find('span', class_='cat cat-play-for-free')
        top_tag = div.find('span', class_='cat cat-top-seller')
        deal_tag = div.find('span', class_=['cat cat-daily-deal','cat cat-weekend-deal','cat cat-midweek-deal','cat cat-week-long-deal','cat cat-introductory-offer'])
        history_tag = div.find('span', class_=['highest-discount-major','highest-discount'])
    else:
        free_tag = np.nan
        top_tag = np.nan
        deal_tag = np.nan
        history_tag = np.nan

    # Buscando a tag <td> e suas siblings
    discount_tag = name.find_next('td', class_=['price-discount-major','price-discount',''])

    # Adicionando as informações ao dicionário
    dic['name'] = name.text
    dic['play_for_free'] = 'yes' if free_tag is not None else 'no'
    dic['top_seller'] = 'yes' if top_tag is not None else 'no'
    dic['deal'] = deal_tag.text if deal_tag is not None else np.nan
    dic['history'] = history_tag.text if history_tag is not None else np.nan
    dic['discount'] = discount_tag.text if discount_tag is not None else np.nan
    dic['price'] = discount_tag.find_next_sibling('td').text if discount_tag is not None else np.nan
    dic['rating'] = discount_tag.find_next_sibling('td').find_next_sibling('td').text if discount_tag is not None else np.nan
    dic['ends in'] = discount_tag.find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').text if discount_tag is not None else np.nan
    dic['started'] = discount_tag.find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').text if discount_tag is not None else np.nan
    dic['release'] = discount_tag.find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').find_next_sibling('td').text if discount_tag is not None else np.nan

    # Adicionando o dicionário à lista de dados
    data.append(dic)

In [5]:
# Criando um Dataframe a partir da lista

df = pd.DataFrame(data)

In [6]:
df

,name,play_for_free,top_seller,deal,history,discount,price,rating,ends in,started,release
0,MELTY BLOOD: TYPE LUMINA,yes,no,NaN,new historical low,-50%,"R$ 46,99",84.79%,in 12 days,2 days ago,Sep 2021
1,Two Point Campus,yes,no,NaN,new historical low,-50%,"R$ 77,75",83.42%,in 12 days,2 days ago,Aug 2022
2,FOR HONOR™,yes,no,NaN,NaN,-80%,"R$ 8,99",68.64%,in 5 days,2 days ago,Feb 2017
3,Shadows of Doubt,no,yes,NaN,new historical low,-20%,"R$ 47,99",88.14%,in 12 days,2 days ago,Apr 2023
4,Deadlink,no,yes,Daily Deal,new historical low,-25%,"R$ 28,49",87.06%,in 6 days,2 days ago,Jul 2023
...,...,...,...,...,...,...,...,...,...,...,...
4702,ALIEN WAR 2 DOGFIGHT,no,no,Introductory Offer,new historical low,-10%,"R$ 23,84",—,,,Jul 2023
4703,ToS EZ Slideshow,no,no,Introductory Offer,new historical low,-10%,"R$ 3,14",—,,,Jul 2023
4704,Bombard,no,no,Introductory Offer,new historical low,-10%,"R$ 15,30",—,,,Jul 2023
4705,Dreaming Chicken,no,no,Introductory Offer,new historical low,-10%,"R$ 15,29",—,,,Jul 2023


In [7]:
#Realizando uma remoção de valores '—' para evitar problemas
df.replace("—", np.nan, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           4707 non-null   object
 1   play_for_free  4707 non-null   object
 2   top_seller     4707 non-null   object
 3   deal           2066 non-null   object
 4   history        2856 non-null   object
 5   discount       4707 non-null   object
 6   price          4707 non-null   object
 7   rating         4566 non-null   object
 8   ends in        4536 non-null   object
 9   started        4707 non-null   object
 10  release        4699 non-null   object
dtypes: object(11)
memory usage: 404.6+ KB


Agora começa a limpeza ;)

Observei que o banco de dados apresenta a maioria das colunas preenchidas com valores não nulos. No entanto, todas as colunas são do tipo 'object', o que pode não ser o ideal para futuras análises.

Portanto, a fim de tornar o conjunto de dados mais apropriado para avaliações futuras, realizarei um tratamento dos dados. Este processo irá assegurar um dataset mais flexível e adequado para análises posteriores.

In [9]:
# Fazendo uma cópia de segurança
steam = df.copy()

In [10]:
# Coluna price
steam['price'] = steam['price'].str.replace('R$ ', '', regex=False)
steam['price'] = steam['price'].str.replace(',', '.')
steam['price'] = steam['price'].astype(float)

# Coluna rating
steam['rating'] = steam['rating'].str.replace('%', '', regex=False)
steam['rating'] = steam['rating'].astype(float) / 100 

# Coluna discount
steam['discount'] = steam['discount'].str.replace('%', '', regex=False)
steam['discount'] = steam['discount'].str.replace('-', '', regex=False)
steam['discount'] = steam['discount'].astype(float) / 100 

# Coluna history
steam['history'] = steam['history'].str.strip('\n')

# Coluna release
steam['release'] = pd.to_datetime(steam['release'], format='%b %Y')

# Colunas started e ends in

steam['ends in'] = steam['ends in'].replace('', np.nan)
steam['started'] = steam['started'].replace('', np.nan)

A coluna 'history' contém informações sobre os preços dos jogos, especificamente se o valor promocional atual é o menor já registrado ou não. No entanto, se o valor atual não é o menor já registrado, a coluna 'history' indica qual foi esse valor mais baixo. Dado que esta é uma coluna do tipo 'object', é desafiador extrair informações devido à diversidade de entradas.

Para facilitar a análise, uma nova coluna que registre o menor preço foi criada. Se o preço atual for o mais baixo já registrado, esse valor será adicionado à nova coluna 'lowest'. Se houver registro de um preço ainda mais baixo, então esse valor será adicionado à coluna 'lowest'.

Em seguida, a padronização de todas as entradas na coluna 'history' que fazem referência a 'all-time low' para uma entrada única e padrão foi efetuada. Isso permite filtrar todos os jogos que estão em promoção, mas ainda não atingiram seu menor valor histórico.

Além disso, a nova coluna 'lowest' permite realizar análises sobre os preços mais baixos já registrados.

In [11]:
# Criando uma nova coluna para armazenar os dados de menores preços
import re

def lowest(row):
    if row['history'] == 'new historical low':
        return row['price']
    else:
        match = re.search(r'R\$ (\d+,\d+)', str(row['history']))
        if match:
            value = match.group(1)
            return float(value.replace(',', '.')) 
        else:
            return np.nan

steam['lowest_price'] = steam.apply(lowest, axis=1)

In [12]:
# com isso posso limpar a coluna history para que fique mais fácil de trabalhar

def replace_all_time(value):
    if str(value).strip().startswith("all"):
        return "not the lowest price"
    else:
        return value

steam['history'] = steam['history'].apply(replace_all_time)

In [13]:
steam

,name,play_for_free,top_seller,deal,history,discount,price,rating,ends in,started,release,lowest_price
0,MELTY BLOOD: TYPE LUMINA,yes,no,NaN,new historical low,0.50,46.99,0.8479,in 12 days,2 days ago,2021-09-01,46.99
1,Two Point Campus,yes,no,NaN,new historical low,0.50,77.75,0.8342,in 12 days,2 days ago,2022-08-01,77.75
2,FOR HONOR™,yes,no,NaN,NaN,0.80,8.99,0.6864,in 5 days,2 days ago,2017-02-01,NaN
3,Shadows of Doubt,no,yes,NaN,new historical low,0.20,47.99,0.8814,in 12 days,2 days ago,2023-04-01,47.99
4,Deadlink,no,yes,Daily Deal,new historical low,0.25,28.49,0.8706,in 6 days,2 days ago,2023-07-01,28.49
...,...,...,...,...,...,...,...,...,...,...,...,...
4702,ALIEN WAR 2 DOGFIGHT,no,no,Introductory Offer,new historical low,0.10,23.84,NaN,NaN,NaN,2023-07-01,23.84
4703,ToS EZ Slideshow,no,no,Introductory Offer,new historical low,0.10,3.14,NaN,NaN,NaN,2023-07-01,3.14
4704,Bombard,no,no,Introductory Offer,new historical low,0.10,15.30,NaN,NaN,NaN,2023-07-01,15.30
4705,Dreaming Chicken,no,no,Introductory Offer,new historical low,0.10,15.29,NaN,NaN,NaN,2023-07-01,15.29


In [14]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4707 entries, 0 to 4706
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           4707 non-null   object        
 1   play_for_free  4707 non-null   object        
 2   top_seller     4707 non-null   object        
 3   deal           2066 non-null   object        
 4   history        2856 non-null   object        
 5   discount       4707 non-null   float64       
 6   price          4707 non-null   float64       
 7   rating         4566 non-null   float64       
 8   ends in        242 non-null    object        
 9   started        249 non-null    object        
 10  release        4699 non-null   datetime64[ns]
 11  lowest_price   2856 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 441.4+ KB


In [15]:
# Salvando

steam.to_csv('steamDB_clean_5K.csv', index=False)

Concluí assim o nosso processo de limpeza de dados, gerando um arquivo CSV com o banco de dados atualizado e pronto para análise. Vale mencionar que era possível realizar outros tratamentos de dados, como a remoção das colunas 'end in' e 'started', que possuem uma grande quantidade de entradas nulas, ou a remoção da coluna 'play_for_free', que embora não contenha valores nulos, tem uma predominância de um tipo específico de entrada.

No entanto, a decisão de realizar esses tratamentos adicionais depende inteiramente do tipo de análise que se deseja realizar nos dados. Como o objetivo era conduzir uma limpeza preliminar dos dados, esses passos adicionais ficam à critério do pesquisador, que pode personalizar o conjunto de dados de acordo com as necessidades de sua investigação específica.